In [1]:
import os
api_key = os.environ['Groq_APIKey']



In [2]:
import glob
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

# Path to all .md files in the folder
markdown_path = "CollegeInfo/allinfo.md"
loader = UnstructuredMarkdownLoader(markdown_path)

data = loader.load()
assert len(data) == 1
assert isinstance(data[0], Document)
readme_content = data[0].page_content
print(readme_content[:250])

#College Info for Engineering Colleges

College 1: BMS College of Engineering

BMS College Overview Duration 4 years

Course Level UG Degree

Mode of Course Full Time

Total Tuition Fees INR 18.00 Lakh

Type of Univers ity Private,Autonomous

BMS Col


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(data)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Load a local embedding model from Hugging Face
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create your vectorstore
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)


C:\Users\rohan\AppData\Local\Temp\ipykernel_29356\3087069557.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\rohan\miniconda3\envs\LangchainRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(vectorstore._collection())

TypeError: 'Collection' object is not callable

: 

In [6]:
retriever = vectorstore.as_retriever()

In [7]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

c:\Users\rohan\miniconda3\envs\LangchainRAG\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [10]:
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)


In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    model="llama3-70b-8192",  # or use "llama3-70b-8192"
    openai_api_key="",
    openai_api_base="https://api.groq.com/openai/v1"
)


C:\Users\rohan\AppData\Local\Temp\ipykernel_29356\4254395850.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [12]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

retriever =vectorstore.as_retriever()

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [17]:
response = rag_chain.invoke({"query": "In JSON format, Recommend 3 engineering courses offered by engineering colleges for a student who's intersted in maths and physics and has a general rank 0f 1500, calculate and include an average the cutoff pf the course you have selected. If any data is ambiguous, leave it and answer the rest of the question instead"})
print(response['result'])


Based on the provided data, here are three engineering course recommendations for a student interested in maths and physics with a general rank of 1500:

```
{
  "Course Recommendations": [
    {
      "Course Name": "B.E. in Computer Science and Engineering",
      "College": "MSRIT College",
      "Cutoff Rank (General)": 1125 (2022), 1171 (2023), 1626 (2024),
      "Average Cutoff Rank": 1311
    },
    {
      "Course Name": "B.Tech. in Electronics and Communication Engineering",
      "College": "PES College",
      "Cutoff Rank (General)": 16146 (2022), 3803 (2023), 8391 (2024),
      "Average Cutoff Rank": 9250
    },
    {
      "Course Name": "B.E. in Electrical and Electronics Engineering",
      "College": "MSRIT College",
      "Cutoff Rank (General)": 9893 (2022), 7763 (2023), 9339 (2024),
      "Average Cutoff Rank": 8998
    }
  ]
}
```

Note: The average cutoff rank is calculated by taking the average of the cutoff ranks for the three years (2022, 2023, and 2024).
